<a href="https://colab.research.google.com/github/kuihao/Learning-record__Machine-learning/blob/main/Learning-record__Machine-learning/CNN/Keras_%E8%AA%B2%E5%A0%82%E6%89%8B%E5%AF%AB%E6%95%B8%E5%AD%97%E8%BE%A8%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 練習與調整過程 Code V.1\~V.8

In [ ]:
!pip install keras

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.datasets import mnist

In [ ]:
#======測試========#
# 查看 MNIST 資料集
#==================#

# 使用 Keras 的函式讀入資料集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 查看原始訓練資料 Input: 
print(x_test.shape) # Training Data 共六萬筆，每一筆是 28*28 的矩陣，因此維度是 3
print("--------------------")
x_train[0]

# 查看原始訓練資料 Output (Label): 
print(y_train.shape) # Training Labels 亦為六萬筆，每一筆是一維資料，顯示手寫數字的對應值
print("--------------------")
y_train[0]

In [ ]:
# 製作匯入資料的函式，內部同時進行資料預處理(維度轉換、資料刪減、增加雜訊、隨機排序、資料正歸化、)
def load_data():  # categorical_crossentropy
    # 透過 Keras Function 讀入資料集
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    # 資料預處理
    ## 減少訓練資料：原本的資料集有60,000筆訓練資料，此處將訓練資料限制為10,000筆
    number = 10000
    x_train = x_train[0:number] 
    y_train = y_train[0:number]
    
    ## 維度轉換，將二維矩陣(Image:28*28)，改成一維陣列(Array:28*28=784)
    x_train = x_train.reshape(number, 28 * 28) 
    x_test = x_test.reshape(x_test.shape[0], 28 * 28)
    ## 以符點數變更資料型態
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    ## 類別資料編碼：使用Keras的One-Hot Ecoding函數 np_utils.to_categorical
    ## 將原始標籤的連續資料型態(Class Vector) 轉變成類別資料型態(Class Matrix)
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)

    ## 給測試資料加入雜訊
    x_test = np.random.normal(x_test)  

    ## 資料標準化
    x_train = x_train / 255
    x_test = x_test / 255

    return (x_train, y_train), (x_test, y_test)

In [ ]:
# 載入手寫數字辨識資料 load training data and testing data
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
# 測試
print(x_train.shape) # Training Data 共一萬筆，每一筆是由 784 維 Vector 所組成
print("--------------------")
x_train[0] # 查看第一個 Vector 的內容，這是一張手寫數字圖片

In [ ]:
# 測試
print(y_train.shape) # 
print("--------------------")
y_train[0]

In [ ]:
    # [Code V1] -- Tran失敗 
    #建立一個線性模型
    model= Sequential()

    # 建立 Hidden Layer 同時加入輸入層
    model.add(Dense(input_dim=28 * 28, units=689, activation='sigmoid'))
    model.add(Dense(units=700, activation='sigmoid'))
    model.add(Dense(units=10, activation='softmax'))

    # 設定 Loss Function
    model.compile(loss='mse',
          optimizer=SGD(lr=0.1), metrics=['accuracy'])
    
    # Train Model
    model.fit(x_train, y_train, batch_size=100, epochs=20)

    # Evaluate the model and output the accuracy
    result_train = model.evaluate(x_train, y_train)
    #result_test = model.evaluate(x_test, y_test)
    print('Train Acc:', result_train[1])
    #print('Test Acc:', result_test[1])

**Tran 失敗時該做的事情：**
  1.   先設法使 Traning Set 達到好的結果 (Deep Learning 很容易Tran壞掉)
    *   Debug: 此處 Loss Function 設得不好，線性平均距離的想法在此不適用，宜改為Cross Entropy 的方法(應用於分類問題，預測結果與實際結果都是假設為分布/抽樣/機率，兩個結果的鐘型曲線模型的重疊越多，理論上未知的不確定因素(Entropy 熵--ㄕㄤ(輸入法 ㄉㄧ))會越小；反之若差距很大則讓Entropy會越大。因此Cross Entropy的方法就是透過求Cross Entropy算式的最小值來找到 **能產生最佳的預測結果的模型**)





In [ ]:
    # [Code V2] 修改 Loss Function 使用 Cross Entropy
    #建立一個線性模型
    model= Sequential()

    # 建立 Hidden Layer 同時加入輸入層
    model.add(Dense(input_dim=28 * 28, units=689, activation='sigmoid'))
    model.add(Dense(units=700, activation='sigmoid'))
    model.add(Dense(units=10, activation='softmax'))

    # 設定 Loss Function
    model.compile(loss='categorical_crossentropy',
          optimizer=SGD(lr=0.1), metrics=['accuracy'])
    
    # Train Model
    model.fit(x_train, y_train, batch_size=100, epochs=20)

    # Evaluate the model and output the accuracy
    result_train = model.evaluate(x_train, y_train)
    #result_test = model.evaluate(x_test, y_test)
    print('Train Acc:', result_train[1])
    #print('Test Acc:', result_test[1])

**Tran 失敗時該做的事情：**
1.   先設法使 Traning Set 達到好的結果 --- OK
2.   Batch Size 設定要夠大但不能過大
  *   Batch Size夠大，程式才會啟用GPU做平行運算
  *   Batch Size太大，GPU硬體不夠多，且一次拿一堆資料做運算則SGD每次計算的隨機性就變小，更新曲線變得太小、太平滑就容易卡在高原難以前進





In [ ]:
    # [Code V3] 調整看看 Batch Size

    # =========== #
    # batch_size=100 得到 accuracy: 0.90 似乎不錯
    # batch_size=10000 快速得到 accuracy: 0.11 看來毀了，批次量太大而卡在高原
    # batch_size=1 更新過慢
    # =========== #

    #建立一個線性模型
    model= Sequential()

    # 建立 Hidden Layer 同時加入輸入層
    model.add(Dense(input_dim=28 * 28, units=689, activation='sigmoid'))
    model.add(Dense(units=700, activation='sigmoid'))
    model.add(Dense(units=10, activation='softmax'))

    # 設定 Loss Function
    model.compile(loss='categorical_crossentropy',
          optimizer=SGD(lr=0.1), metrics=['accuracy'])
    
    # Train Model
    model.fit(x_train, y_train, batch_size=10000, epochs=20) 

    # Evaluate the model and output the accuracy
    result_train = model.evaluate(x_train, y_train)
    #result_test = model.evaluate(x_test, y_test)
    print('Train Acc:', result_train[1])
    #print('Test Acc:', result_test[1])

**Tran 失敗時該做的事情：**
1.   先設法使 Traning Set 達到好的結果 --- OK
2.   Batch Size 設定要夠大但不能過大 --- OK
3.   Layer 層數數量不宜過大
  * 層數過多會導致 Gradient Vanishing

In [ ]:
    # [Code V4] 試試修改層數

    # =======
    # Hidden Layer Numbers = 10
    # 得到 Accuracy: 0.10
    # 又卡住了，更新的微分值趨近於零，幾乎不更新了 
    # =======

    #建立一個線性模型
    model= Sequential()

    # 建立 Hidden Layer 同時加入輸入層
    model.add(Dense(input_dim=28 * 28, units=689, activation='sigmoid'))
    # 重複建立10層隱藏層
    for i in range(10):
      model.add(Dense(units=700, activation='sigmoid'))
    model.add(Dense(units=10, activation='softmax'))

    # 設定 Loss Function
    model.compile(loss='categorical_crossentropy',
          optimizer=SGD(lr=0.1), metrics=['accuracy'])
    
    # Train Model
    model.fit(x_train, y_train, batch_size=100, epochs=20)

    # Evaluate the model and output the accuracy
    result_train = model.evaluate(x_train, y_train)
    #result_test = model.evaluate(x_test, y_test)
    print('Train Acc:', result_train[1])
    #print('Test Acc:', result_test[1])

**Tran 失敗時該做的事情：**
1.   先設法使 Traning Set 達到好的結果 --- OK
2.   Batch Size 設定要夠大但不能過大 --- OK
3.   Layer 層數數量不宜過大 --- OK
4.   修改 Activation Function
  * 有趣的是當維持隱藏層為10層，直接改 Activation 為 ReLu 也能練起來

In [ ]:
    # [Code V5] 修改Activation Function為ReLu

    # =======
    # 保持隱藏層=10，使用 ReLu 為激勵函數
    # Traning Set 得到 Accuracy: 0.99 -- 驚人的訓練成功
    # Testing Set 得到 Accuracy: 0.95 -- 似乎不錯的預測結果
    #
    # 隱藏層改回1，使用 ReLu 為激勵函數
    # Traning Set 得到 Accuracy: 0.99
    # Testing Set 得到 Accuracy: 0.95
    # =======


    #建立一個線性模型
    model= Sequential()

    # 建立 Hidden Layer 同時加入輸入層
    model.add(Dense(input_dim=28 * 28, units=689, activation='relu'))
    # 重複建立10層隱藏層
    #for i in range(10):
    #  model.add(Dense(units=700, activation='relu'))
    #建立1層隱藏層
    model.add(Dense(units=700, activation='relu'))
    model.add(Dense(units=10, activation='softmax'))

    # 設定 Loss Function
    model.compile(loss='categorical_crossentropy',
          optimizer=SGD(lr=0.1), metrics=['accuracy'])
    
    # Train Model
    model.fit(x_train, y_train, batch_size=100, epochs=20)

    # Evaluate the model and output the accuracy
    result_train = model.evaluate(x_train, y_train)
    print('Train Acc:', result_train[1])
    result_test = model.evaluate(x_test, y_test)
    print('Test Acc:', result_test[1])

**Tran 失敗時該做的事情：**
1.   先設法使 Traning Set 達到好的結果 --- OK
2.   **嘗試標準化(Normalizing)** 
  * 前面自製的 Load 函數裡有對圖片的每個資料(Pixel)做標準化，原本的值是範圍0\~255的整數，代表圖片像素的灰階值，Load 函數裡將資料除以255，使得資料的值變成範圍0\~1之間的小數，1代表塗黑、0代表空白，(猜測：形成如同二元機率分布，而小數就代表接近兩端類別的距離。
3.   Batch Size 設定要夠大但不能過大 --- OK
4.   Layer 層數數量不宜過大 --- OK
5.   修改 Activation Function

In [ ]:
    # [Code V6] 若沒標準化則會訓練不起來

    # =======
    # 若沒作資料標準化
    # Traning Set 得到 Accuracy: 0.09
    # =======

    def load_data():  # categorical_crossentropy
      (x_train, y_train), (x_test, y_test) = mnist.load_data()
      number = 10000
      x_train = x_train[0:number] 
      y_train = y_train[0:number]
      x_train = x_train.reshape(number, 28 * 28) 
      x_test = x_test.reshape(x_test.shape[0], 28 * 28)
      x_train = x_train.astype('float32')
      x_test = x_test.astype('float32')
      y_train = np_utils.to_categorical(y_train, 10)
      y_test = np_utils.to_categorical(y_test, 10)
      x_test = np.random.normal(x_test)  

      # 取消標準化
      #x_train = x_train / 255
      #x_test = x_test / 255

      return (x_train, y_train), (x_test, y_test)
    
    # 重讀資料
    (x_train, y_train), (x_test, y_test) = load_data()

    #建立一個線性模型
    model= Sequential()

    # 建立 Hidden Layer 同時加入輸入層
    model.add(Dense(input_dim=28 * 28, units=689, activation='relu'))
    # 重複建立10層隱藏層
    for i in range(10):
      model.add(Dense(units=700, activation='relu'))
    model.add(Dense(units=10, activation='softmax'))

    # 設定 Loss Function
    model.compile(loss='categorical_crossentropy',
          optimizer=SGD(lr=0.1), metrics=['accuracy'])
    
    # Train Model
    model.fit(x_train, y_train, batch_size=100, epochs=20)

    # Evaluate the model and output the accuracy
    #result_train = model.evaluate(x_train, y_train)
    result_test = model.evaluate(x_test, y_test)
    #print('Train Acc:', result_train[1])
    print('Test Acc:', result_test[1])

**Tran 失敗時該做的事情：**
1.   先設法使 Traning Set 達到好的結果 --- OK
2.   嘗試標準化(Normalizing)
3.   Batch Size 設定要夠大但不能過大 --- OK
4.   Layer 層數數量不宜過大 --- OK
5.   修改 Activation Function
6.   修改 Optimizer (Traning Gradient Descenr 的方法)
  * 改為 Adam 會發現剛開始的正確率飆升地更快 (但通常伴隨難以收斂的問題)

In [ ]:
    # [Code V7] 更改 Traning Gradient Descenr 的策略

    # =======
    # 改為 Adam
    # Traning Set 得到 Accuracy: 1 (100%)，
    #   並且 Epoch 1->2 時，Acc: 0.77->0.95， 
    #   相較使用 SGD 時，Acc:0.67->0.89， 顯然 Adam 爬升更快速
    # Testing Set 得到 Accuracy: 0.961
    # =======

    # 建立一個線性模型
    model= Sequential()

    # 建立 Hidden Layer 同時加入輸入層
    model.add(Dense(input_dim=28 * 28, units=689, activation='relu'))

    # 建立隱藏層
    model.add(Dense(units=700, activation='relu'))

    # 建立輸出層
    model.add(Dense(units=10, activation='softmax'))

    # 設定 Loss Function 並使用 Adam
    model.compile(loss='categorical_crossentropy',
          optimizer='Adam', metrics=['accuracy'])
    
    # Train Model
    model.fit(x_train, y_train, batch_size=100, epochs=20)

    # Evaluate the model and output the accuracy
    result_train = model.evaluate(x_train, y_train)
    print('Train Acc:', result_train[1])
    result_test = model.evaluate(x_test, y_test)
    print('Test Acc:', result_test[1])

**Tran 失敗時該做的事情：**
1.   先設法使 Traning Set 達到好的結果 --- OK
2.   嘗試標準化(Normalizing)
3.   Batch Size 設定要夠大但不能過大 --- OK
4.   Layer 層數數量不宜過大 --- OK
5.   修改 Activation Function
6.   修改 Optimizer (Traning Gradient Descenr 的方法)
7.   當發生 Overfitting 時， 可以使用 Dropout
  * 課程範例是在此時給 Testing Data 加入 Noise，然後 Model 就 Overfitting， 預測不準
  * 幸運的是，本 Code 一開始就不小心加入 Noise 作訓練， 正確率已達 0.965，但仍能看看 Dropout 的效果 (應該會不佳?)
  * 注意使用時機: Overfitting，且如果預測結果差異很大，Dropout Rate 可以設 >0.5，而通常是先試 Rate=0.5 
  * 注意使用方法: Dropout 要加在每個 Hidden Layer 後面 (包含輸入層，不包含輸出層)
  * 注意分析結果: Testing 預測結果理應變好，且 **Traning 結果反而會變差**，綁住 Network 手腳 (去掉一些 Node(Neuron))

In [ ]:
    # [Code V8] 嘗試加入 Dropout

    # =======
    # Traning Set 得到 Accuracy: 0.999
    # Testing Set 得到 Accuracy: 0.964
    # Traning Accuracy 從 1 下降為 0.999
    # Testing Accuracy 從 0.961 上升至 0.965 
    # =======

    # 建立一個線性模型
    model= Sequential()

    # 建立 Hidden Layer 同時加入輸入層
    model.add(Dense(input_dim=28 * 28, units=689, activation='relu'))
    model.add(Dropout(0.5)) # 加 Dropout
    # 建立隱藏層
    model.add(Dense(units=700, activation='relu'))
    model.add(Dropout(0.5)) # 加 Dropout
    # 建立輸出層
    model.add(Dense(units=10, activation='softmax'))

    # 設定 Loss Function 並使用 Adam
    model.compile(loss='categorical_crossentropy',
          optimizer='Adam', metrics=['accuracy'])
    
    # Train Model
    model.fit(x_train, y_train, batch_size=100, epochs=20)

    # Evaluate the model and output the accuracy
    result_train = model.evaluate(x_train, y_train)
    print('Train Acc:', result_train[1])
    result_test = model.evaluate(x_test, y_test)
    print('Test Acc:', result_test[1])



---



# 最終完整的神經網路 Code

In [ ]:
# 引入 Keras 及其他必要套件
import numpy as np
from keras.models import Sequential
from keras.models import load_model 
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.datasets import mnist

In [ ]:
# 製作匯入資料的函式，內部同時進行資料預處理(維度轉換、資料刪減、增加雜訊、隨機排序、資料正歸化、)
def load_data():  # categorical_crossentropy
    # 透過 Keras Function 讀入資料集
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    # 資料預處理
    ## 減少訓練資料：原本的資料集有60,000筆訓練資料，此處將訓練資料限制為10,000筆
    number = 10000
    x_train = x_train[0:number] 
    y_train = y_train[0:number]
    
    ## 維度轉換，將二維矩陣(Image:28*28)，改成一維陣列(Array:28*28=784)
    x_train = x_train.reshape(number, 28 * 28) 
    x_test = x_test.reshape(x_test.shape[0], 28 * 28)
    ## 以符點數變更資料型態
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    ## 類別資料編碼：使用Keras的One-Hot Ecoding函數 np_utils.to_categorical
    ## 將原始標籤的連續資料型態(Class Vector) 轉變成類別資料型態(Class Matrix)
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)

    ## 給測試資料加入雜訊
    x_test = np.random.normal(x_test)  

    ## 資料標準化
    x_train = x_train / 255
    x_test = x_test / 255

    return (x_train, y_train), (x_test, y_test)

In [ ]:
# 載入手寫數字辨識資料 load training data and testing data
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
# [Code V9] 完成的 Code 
# (Code在輸出預測的部分好像有點問題，因為 Testing 時分開不能使用到 Dropout)
# 應該要 Tran 好 Model 後下參數，Testing 時不再加 Dropout

# 建立一個線性模型
model = Sequential()

# 建立 Hidden Layer 同時加入輸入層
model.add(Dense(input_dim=28 * 28, units=500, activation='relu'))
model.add(Dropout(0.5)) # Dropout放棄一些神經元，Overfitting 時使用

## 建立輸出層
model.add(Dense(units=500, activation='relu'))
model.add(Dropout(0.5))# Dropout放棄一些神經元，Overfitting 時使用

# 建立輸出層(維度需要對應Labels)
model.add(Dense(units=10, activation='softmax'))

# 設定 Loss Function，使用 CrossEntropy
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# Train Model
model.fit(x_train, y_train, batch_size=100, epochs=20)

# Save Neuron's Parameter
model.save('./sample_data/File_SaveModel/Model_Parameter_useDropout.h5')
print('Save Parameter Successfully')

# Evaluate the model and output the accuracy
result_train = model.evaluate(x_train, y_train)
print('Train Acc:', result_train[1])

In [ ]:
# 測試模型

## 載入參數
model_rebuild = load_model('/content/sample_data/File_SaveModel/Model_Parameter_useDropout.h5')

## 測試模型
score = model_rebuild.evaluate(x_test, y_test)

## 印出結果
print('---測試結果如下----')
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

In [ ]:
# 視覺化
import matplotlib.pyplot as plt

# 定義顯示圖片的函數，參數(images:輸入Model的圖像特徵值, labels:輸出Model的標籤, 
#             prediction:預測值, idx:顯示圖像的起始編號, num:顯示圖像的總數)
def plot_images_labels_prediction(images,labels,prediction,idx,num=10):
  # 取得 pyplot 物件 -- Figure(畫布)
  fig=plt.gcf() 

  # 設定畫布大小為 12*14
  fig.set_size_inches(12, 14) 

  # 限制最多顯示 25 個子圖
  if num > 25: num=25 
  
  # 依序顯示 num 個子圖
  for i in range(0, num): 
    # 建立 5*5 個子圖中的第 i+1 個
    ax=plt.subplot(5, 5, i+1) 
    
    # 顯示子圖
    ax.imshow(images[idx], cmap='binary') 
    title="label=" + str(labels[idx])
    
    # 若有預測值就加入標題中
    if len(prediction) > 0: 
      title += ",predict=" + str(prediction[idx])

    # 設定標題
    ax.set_title(title, fontsize=10) 
    # 不顯示 x 軸刻度
    ax.set_xticks([]); 
    # 不顯示 y 軸刻度
    ax.set_yticks([]); 
    # 樣本序號增量 1
    idx += 1 
  # 繪製圖形  
  plt.show() 

In [ ]:
# 進行預測並繪出圖像
## 紀錄預測值
prediction = model.predict_classes(x_test)

## 反轉(特徵值從陣列轉回矩陣、類別資料轉回連續資料)
Image = x_test.reshape(10000, 28, 28)
Label = y_test.argmax(1)

plot_images_labels_prediction(Image, Label, prediction, 300, 25)